In [1]:
from lxml import html
import requests
from pprint import pprint
import re
import pandas as pd
from pymongo import MongoClient
import datetime

header = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 YaBrowser/20.7.3.101 Yowser/2.5 Yptp/1.23 Safari/537.36'}

params_dict = {"mail": {"host": "https://news.mail.ru",   # главная ссылка
                    "items": "//div[@class='js-module']//a[contains(@class, 'photo_full')]|//ul[@name='clb20268353']/li[@class='list__item']", # Контейнер с новостями
                    "link": ".//@href",
                    "name": ".//text()",
                    "time_news" : "//span[@class='note']/span[@datetime]/@datetime",
                    "name_source": "//span[@class='note']/a//text()",
                    "link_source": "//span[@class='note']/a/@href"
                    },
                "yandex": {"host": "https://yandex.ru/news",
                        "items": "//div[contains(@class, 'news-top-rubric-stories')][1]//article",
                        "link": ".//a[@class='news-card__link']/@href",
                        "name": ".//a[@class='news-card__link']//text()",
                        "name_source": ".//span[@class='mg-card-source__source']//text()",
                        "link_source": "//a[contains(@class, 'news-story__subtitle')]/@href",
                        "time_news": ".//span[@class='mg-card-source__time']//text()"
                        },
                "lenta": {"host": "https://lenta.ru",
                         "items": "//div[@class='span4']/div[contains(@class, 'item')]//time",
                         "link": "./../@href",
                         "name": "./../text()",
                         "name_source": "LENTA.RU",
                         "time_news": "./@datetime"
                         }
				}

def full_link_func(host, arg_1, arg_2):
    if 'http' in arg_1:
        arg_2 = arg_1
    elif '/news' in arg_1:
        arg_2 = host + arg_1[5:]
    else: 
        arg_2 = host + arg_1
    return arg_2

def d_t_func(time_news):
    if '+03:00' in time_news:
        time_news = datetime.datetime.strptime(time_news[:19], "%Y-%m-%dT%H:%M:%S")
        time_news = time_news.strftime('%Y-%m-%d %H:%M')
    elif "вчера" in time_news:
        date_news = str(datetime.date.today() + datetime.timedelta(-1))
        time_news = datetime.datetime.strptime(date_news + time_news[8:], '%Y-%m-%d%H:%M')
        time_news = time_news.strftime('%Y-%m-%d %H:%M')
    elif time_news == None:
        time_news = None
    elif ',' in time_news:
        t_split = time_news.split(', ')
        t = str(datetime.datetime.strptime(t_split[0], ' %H:%M').time())
        d = str(datetime.date.today())
        time_news = datetime.datetime.strptime(d+t, '%Y-%m-%d%H:%M:%S')
        time_news = time_news.strftime('%Y-%m-%d %H:%M')
    else:
        date_news = str(datetime.date.today())
        time_news = datetime.datetime.strptime(date_news + time_news, '%Y-%m-%d%H:%M')
        time_news = time_news.strftime('%Y-%m-%d %H:%M')
    return time_news
         

def parse_news(source):
    response = requests.get(params_dict[source]['host'], headers=header)
    dom = html.fromstring(response.text)
    items = dom.xpath(params_dict[source]['items'])
    count = 1
    news = []
    full_link = None
    for item in items:
        new = {}
        new['count']= count
        link = item.xpath(params_dict[source]['link'])[0]
        full_link = full_link_func(params_dict[source]['host'], link, full_link)
        new['full_link'] = full_link
        new['name'] = item.xpath(params_dict[source]['name'])[0].replace('\xa0',' ')
        if source == 'mail':        
            new_response = requests.get(full_link, headers=header)
            new_dom = html.fromstring(new_response.text)
            time_news = new_dom.xpath(params_dict[source]['time_news'])[0]
            new['time_news'] = d_t_func(time_news)
            new['name_source'] = new_dom.xpath(params_dict[source]['name_source'])[0]
            new['link_source'] = new_dom.xpath(params_dict[source]['link_source'])[0]
        elif source == 'yandex':
            new['name_source'] = item.xpath(params_dict[source]['name_source'])[0]
            new_response = requests.get(full_link, headers=header)
            new_dom = html.fromstring(new_response.text)
            time_news = item.xpath(params_dict[source]['time_news'])[0]
            new['time_news'] = d_t_func(time_news)
            new['link_source'] = new_dom.xpath(params_dict[source]['link_source'])[0]
        else:    
            new['name_source'] = params_dict[source]['name_source']
            time_news = item.xpath(params_dict[source]['time_news'])[0]
            new['time_news'] = d_t_func(time_news)
            new['link_source'] = params_dict[source]['host']
        count += 1
        news.append(new)
    return news    


mail = pd.DataFrame(parse_news('mail'))
yandex = pd.DataFrame(parse_news('yandex'))
lenta = pd.DataFrame(parse_news('lenta'))

news_df = pd.concat([mail, yandex, lenta])

In [2]:
news_df

,count,full_link,name,time_news,name_source,link_source
0,1,https://news.mail.ru/politics/42895375/,Лукашенко: мы договорились с Путиным по задерж...,2020-08-09 12:17,Коммерсантъ,http://www.kommersant.ru
1,2,https://news.mail.ru/politics/42895835/,ЦИК признал состоявшимися выборы президента Бе...,2020-08-09 14:02,Коммерсантъ,http://www.kommersant.ru
2,3,https://sportmail.ru/news/olympics/42895979/,"Родченков заявил, что в США скрывали положител...",2020-08-09 14:16,Lenta.Ru,http://lenta.ru/rubrics/sport/
3,4,https://news.mail.ru/society/42896508/,Медик сравнил опасность нового вируса SFTS и к...,2020-08-09 15:56,РИА Новости,http://www.ria.ru
4,5,https://news.mail.ru/society/42894019/,Назван самый малочисленный народ России,2020-08-09 09:45,Коммерсантъ,http://www.kommersant.ru
5,6,https://news.mail.ru/society/42894375/,Россияне увидят самый яркий звездопад года,2020-08-09 10:20,Погода Mail.ru,http://pogoda.mail.ru/
6,7,https://news.mail.ru/politics/42893074/,В США объяснили переброску войск из Германии н...,2020-08-09 05:58,РИА Новости,http://www.ria.ru
7,8,https://news.mail.ru/society/42896955/,В Лондоне появились дизайнерские пешеходные пе...,2020-08-09 15:48,Новости Mail.ru,https://news.mail.ru
8,9,https://news.mail.ru/society/42789304/,День в истории: 9 августа,2020-08-09 00:05,Новости Mail.ru,https://news.mail.ru
9,10,https://sportmail.ru/news/football-foreign/428...,Роналду близок к уходу в ПСЖ,2020-08-09 13:41,Чемпионат.com,http://www.championat.com/


In [3]:
client = MongoClient('127.0.0.1', 27017)
news_db = client['news_db']
news = news_db.news

In [7]:
news_df.reset_index(inplace=True)
news.insert_many(news_df.to_dict('records'))

In [8]:
for new in news.find({}):
    print(new)

{'_id': ObjectId('5f300347a192161095dd3551'), 'level_0': 0, 'index': 0, 'count': 1, 'full_link': 'https://news.mail.ru/politics/42895375/', 'name': 'Лукашенко: мы договорились с Путиным по задержанным россиянам', 'time_news': '2020-08-09 12:17', 'name_source': 'Коммерсантъ', 'link_source': 'http://www.kommersant.ru'}
{'_id': ObjectId('5f300347a192161095dd3552'), 'level_0': 1, 'index': 1, 'count': 2, 'full_link': 'https://news.mail.ru/politics/42895835/', 'name': 'ЦИК признал состоявшимися выборы президента Белоруссии', 'time_news': '2020-08-09 14:02', 'name_source': 'Коммерсантъ', 'link_source': 'http://www.kommersant.ru'}
{'_id': ObjectId('5f300347a192161095dd3553'), 'level_0': 2, 'index': 2, 'count': 3, 'full_link': 'https://sportmail.ru/news/olympics/42895979/', 'name': 'Родченков заявил, что в США скрывали положительные допинг-пробы', 'time_news': '2020-08-09 14:16', 'name_source': 'Lenta.Ru', 'link_source': 'http://lenta.ru/rubrics/sport/'}
{'_id': ObjectId('5f300347a192161095dd35